# Grassp Centrifugation Workflow Tutorial


*grassp* is a python package that facilitates the analysis of subcellular proteomics data (with an emphasis on graph-based analyses). In this tutorial we will be analyzing subcellular proteomics data produced by differential ultracentrifugation (DC). 



In [ ]:
# Spatial and single cell analysis
import grassp as gr
import scanpy as sc

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Numerical computing and statistics
import numpy as np

## Reading files

We'll load the count matrix into an AnnData object, a data structure that provides multiple compartments for storing annotations and various data representations. For a comprehensive tutorial, refer to the [Getting Started with AnnData](https://anndata.readthedocs.io/en/stable/tutorials/notebooks/getting-started.html) guide.


In [ ]:
# Grassp provides methods to read from common proteomics formats in the io module.
# Here we read a MaxQuant output file.
dc = gr.io.read_maxquant(
    "https://public.czbiohub.org/proteinxlocation/internal/proteinGroups.txt",
    intensity_column_prefixes=["LFQ intensity ", "MS/MS count "],
)

dc.var["subcellular_enrichment"] = dc.var_names.str.split("_").str[-1]
dc.var["subcellular_enrichment"] = dc.var["subcellular_enrichment"].replace(
    "cyt", "Cyt"
)

dc.var["biological_replicate"] = dc.var_names.str.split("_").str[0].str[-1]

The centrifugation data in this tutorial comes from the Elias lab at Stanford University (unpublished as of July 2025). Centrifugation-based subcellular fractionation experiments separate cellular components by spinning samples at increasing speeds (1K, 3K, 5K, 12K, 24K, 80K × g) to isolate organelles and subcellular structures based on their density and size, with the cytoplasmic fraction (Cyt) representing the final supernatant. 

This approach differs from immunoprecipitation (IP) pull-downs, which use antibodies to specifically capture target proteins and their interacting partners.

Although we are loading in the data from our online data repository, grassp comes with several example datasets. The code lines below shows how to load in these data, including arguments for raw or enriched data. 

In [ ]:
# example_load_raw = gr.datasets.hek_dc_2025(enrichment="raw")
# example_load_enr = gr.datasets.hek_dc_2025(enrichment="enriched")

In [ ]:
dc

Let's go through the information printed above:

`n_obs` is the number of "Observations" (i.e. proteins), `n_var` is the number of variables (i.e. pulldowns/fractions).
> AnnData object with n_obs × n_vars = 10224 × 42

Under `obs` we find the metadata for the proteins. Each entry is a column in a pandas DataFrame.
>    obs: 'Protein IDs', 'Majority protein IDs', 'Peptide counts (all)', 'Peptide counts (razor+unique)', 'Peptide counts (unique)', 'Protein names', 'Gene names', 'Fasta headers', 'Number of proteins', 'Peptides', 'Razor + unique peptides', 'Unique peptides', 'Sequence coverage [%]', 'Unique + razor sequence coverage [%]', 'Unique sequence coverage [%]', 'Mol. weight [kDa]', 'Sequence length', 'Sequence lengths', 'Fraction average', 'Fraction 1', 'Fraction 2', 'Fraction 3', 'Q-value', 'Score', 'Intensity', 'iBAQ', 'MS/MS count', 'Only identified by site', 'Reverse', 'Potential contaminant', 'id', 'Peptide IDs', 'Peptide is razor', 'Mod. peptide IDs', 'Evidence IDs', 'MS/MS IDs', 'Best MS/MS', 'Oxidation (M) site IDs', 'Oxidation (M) site positions'

Under `var` we find the metadata for the pulldowns/Fractions.
>   var: 'subcellular_enrichment', 'biological_replicate'

## Preprocessing

### Adding Compartment Annotations
In addition to the bare bones AnnData object, it can be important to add annotations that specify the ground truth subcellular compartments for each sample. These compartment annotations serve as reference labels that define which organelles and cellular structures are expected to be enriched at each centrifugation speed. 

In [ ]:
gr.pp.add_markers(dc, species="hsap")
dc.obs[20:30]

### Adding QC metrics to the metadata
Before performing filtering and transformations, let's add some quality control metrics of the raw data to the metadata, which we can plot later on. 

In [ ]:
gr.pp.calculate_qc_metrics(dc)

### Filtering

In [ ]:
dc_filtered = dc.copy()

`grassp.pp` provides filtering functions to remove low-quality proteins. Here, we filter out proteins that were annotated as contaminants by MaxQuant and then remove proteins that were not at least detected in 2/6 fractions for 4/6 replicates. 

In [ ]:
print("Protein count before filtering: ", dc.shape[0])

contaminant_cols = ["Only identified by site", "Reverse", "Potential contaminant"]
gr.pp.remove_contaminants(dc, filter_columns=contaminant_cols, filter_value="+")
dc_filtered.obs.drop(
    columns=contaminant_cols,
    inplace=True,
)
print("Protein count after contaminant filtering: ", dc.shape[0])

gr.pp.filter_proteins_per_replicate(
    dc_filtered,
    grouping_columns="subcellular_enrichment",
    min_replicates=4,
    min_samples=2,
)
print("Protein count after replicate filtering: ", dc_filtered.shape[0])

### Transformations

#### Normalization (log1p transformation)
Plotting functions like PCA assume normally distributed data, so it's necessary to apply log transformation to the count data to reduce skewness and stabilize variance across the dynamic range of protein abundances.

In [ ]:
dc_filtered.layers["raw_intensities"] = dc_filtered.X.copy()
print(f"DC data before log transforming {dc_filtered.X[:10, :5]}")

dc_filtered.X = np.log1p(dc_filtered.X)
print(f"DC data before imputating {dc_filtered.X[:10, :5]}")
dc_filtered.layers["log_intensities"] = dc_filtered.X.copy()

#### Imputing
Mass spectrometry data contains numerous missing values due to instrument sensitivity thresholds, where proteins below the detection limit are not quantified. 

Imputation addresses these technical limitations by estimating missing values for more comprehensive downstream analysis.
In this case, grassp uses a left-shifted gaussian imputation, although other methods can be chosen. 

In [ ]:
gr.pp.impute_gaussian(dc_filtered, distance=1.8)
print(f"DC data after imputating {dc_filtered.X[:10, :5]}")

Plotting histogram of data distribution before versus after imputation

In [ ]:
plt.hist(
    dc_filtered.X.flatten(), bins=100, alpha=0.5, label="After Imputation"
)  # .flatten() converts the matrix into a 1D array
plt.hist(
    dc_filtered.layers["log_intensities"].flatten(),
    bins=100,
    alpha=0.5,
    label="Before Imputation",
)
plt.legend()


### QC plotting 

#### Plotting transposed PCA to check sample clustering 
In subcellular proteomics we focus on the relationships between proteins, which typically lie in our "observations". However, for QC, one might want to compare samples.
Therefore, transposed PCA allows us to visualize ***sample*** relationships and ensure that biological replicates cluster together as expected.

In [ ]:
dc_T = dc_filtered.T.copy()
dc_T.X = dc_filtered.layers["log_intensities"].T

sc.pp.pca(dc_T)
sc.pl.pca(dc_T, color="subcellular_enrichment", palette="cividis")

#### Violin plots of Log Intensities per Sample
Plottting the distribution of protein intensities across each sample helps to identify any samples with unusual expression patterns or technical issues. As expected, the later fractions and Cytosolic supernatant have fewer proteins.

In [ ]:
plot_df = dc_filtered.to_df(layer="log_intensities")

plt.figure(figsize=(20, 4))
sns.violinplot(plot_df, inner=None)
plt.xticks(rotation=90)
plt.xticks(rotation=90)
plt.tight_layout()  # No fig needed
plt.title(label="Log1p Intensities per Sample")
plt.show()

#### Violin plot of QC metrics per Fraction
In addition to sample-level qc, we can examine quality control metrics at the fraction level, revealing how protein detection rates, total intensities, and dropout percentages vary across different centrifugation speeds and compartments.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(28, 6))
for key, ax in zip(
    ["n_proteins_by_intensity", "log1p_total_intensity", "pct_dropout_by_intensity"],
    axs,
):
    sc.pl.violin(
        dc_T,
        key,
        groupby="subcellular_enrichment",
        size=4,
        rotation=90,
        ax=ax,
        show=False,
    )

### Enrichment (Log Fold Change)
Grassp provides functions to calculate enrichments. The following useful grassp enrichment function offers two enrichment calculation methods:

1. **Log fold change (lfc)** computes the difference between median intensities of the target sample versus all other samples in the same condition, providing a measure of how many times higher or lower protein levels are in the enriched fraction
2. **Proportion** calculates the relative abundance as a fraction of total intensity across target and control samples, indicating what percentage of the protein's total signal comes from the enriched condition

Here we chose the lfc transformation over proportions, because it produces a distribution of values that is closer to a normal distribution. As aforementioned, many downstream tools such as PCA assume normally distributed features.

In [ ]:
dc_filtered_enr = gr.pp.calculate_enrichment_vs_all(
    dc_filtered,
    subcellular_enrichment_column="subcellular_enrichment",
    covariates=["biological_replicate"],
    enrichment_method="lfc",
)

dc_filtered_enr = gr.pp.aggregate_samples(
    dc_filtered_enr, grouping_columns="subcellular_enrichment", agg_func=np.median
)

dc_filtered_enr

## Dimensionality Reduction

### PCA plots 
Having filtered, transformed, and enriched, we can move onto visualization and interpretation! These plots show how proteins cluster in reduced dimensional space based on their intensity patterns across samples, revealing groups of co-localized proteins and identifying potential subcellular localization signatures.

In [ ]:
sc.pp.scale(dc_filtered_enr)
sc.pp.pca(dc_filtered_enr)
sc.pl.pca(
    dc_filtered_enr,
    color="hein2024_gt_component",
    title="DC hein 2024 Ground Truth PCA",
)
sc.pl.pca(
    dc_filtered_enr, color="hein2024_component", title="DC hein 2024 annotated PCA"
)

### UMAPs
While PCA provides a linear dimensionality reduction, UMAP offers a non-linear approach that can better preserve local neighborhood structures and reveal more complex patterns in protein localization data that might be missed by linear methods.

In [ ]:
sc.pp.neighbors(dc_filtered_enr, use_rep="X", n_neighbors=20)
sc.tl.umap(dc_filtered_enr)
sc.pl.umap(
    dc_filtered_enr,
    color="hein2024_gt_component",
    title="DC hein 2024 Ground Truth UMAP",
)
sc.pl.umap(
    dc_filtered_enr, color="hein2024_component", title="DC hein 2024 annotated UMAP"
)


## Compartment Annotation

The central question of subcellular proteomics is to find which cellular compartment each observed protein resides in. One way to annotate proteins with their compartments is to start from a set of ground-truth proteins with known localization and transfer labels to proteins with similar subcellular profiles. For this grassp provides the `knn_annotation` function, that propagates labels across local neighborhoods in the protein-protein neighbor graph.

In [ ]:
gr.tl.knn_annotation(
    dc_filtered_enr, gt_col="hein2024_gt_component", key_added="knn_annotation"
)

sc.pl.umap(dc_filtered_enr, color="knn_annotation", title="KNN Annotation")

After these steps, you will see that new analysis results are stored in various AnnData compartments: PCA components and UMAP coordinates are saved in .obsm, while metadata like search engine parameters and visualization settings are stored in .uns, and protein-protein relationships are captured in .obsp as distance and connectivity matrices. 

>   uns: 'Search_Engine', 'pca', 'hein2024_gt_component_colors', 'hein2024_component_colors', 'neighbors', 'umap'

>   obsm: 'X_pca', 'X_umap'

>   obsp: 'distances', 'connectivities'

In [ ]:
dc_filtered_enr